In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc

In [2]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

# Loading Data From Folders

In [3]:
#Available Datasets

def params(current_case = 'Case5'): 
    
    code_models = {
        'Case1':('EleutherAI/gpt-neo-125M','codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv','EleutherAI-gpt-neo-125M_10000_','callbacks-EleutherAI-gpt-neo-125M_10000_'),
        'Case2':('EleutherAI/gpt-neo-2.7B','codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv','EleutherAI-gpt-neo-2.7B_10000_','callbacks-EleutherAI-gpt-neo-2.7B_10000_'),    
        'Case3':('EleutherAI/gpt-neo-1.3B','codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv','EleutherAI-gpt-neo-1.3B_10000_','callbacks-EleutherAI-gpt-neo-1.3B_10000_'),
        'Case4':('microsoft/CodeGPT-small-py','codesearch_tesbed_microsoft-CodeGPT-small-py_1024_10000.csv','CodeGPT-small-py_10000_','callbacks-CodeGPT-small-py_10000_'),
        'Case5':('microsoft/CodeGPT-small-py-adaptedGPT2','codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv','CodeGPT-small-py-adaptedGPT2_10000_','callbacks-CodeGPT-small-py-adaptedGPT2_10000_'),
        'Case6':('Salesforce/codegen-2B-multi','codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv','Salesforce-codegen-2B-multi_10000_','callbacks-Salesforce-codegen-2B-multi_10000_')
    }
    
    #print(code_models[current_case][1])
    
    return {
            'big_table_path' : '../data/concept_tables/' + code_models[current_case][1],
            'hf_model' :  code_models[current_case][0],
            'model_name': code_models[current_case][2],
            'callbacks' : '../datax/' + code_models[current_case][3],
            'wpe':2048,  #<----[Hyper] 1024 2048
            'outputs': '../datax/outputs/' + 'out_' + code_models[current_case][1]
        }   

In [4]:
#pwd
parameters = params()
parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv',
 'hf_model': 'microsoft/CodeGPT-small-py-adaptedGPT2',
 'model_name': 'CodeGPT-small-py-adaptedGPT2_10000_',
 'callbacks': '../datax/callbacks-CodeGPT-small-py-adaptedGPT2_10000_',
 'wpe': 2048,
 'outputs': '../datax/outputs/out_codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv'}

In [5]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [6]:
data_pd.head(2)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def register_context_middleware(self, *middlew...","[('def', 'def', 'function_definition'), ('regi...","[(4299, 'def', 'function_definition'), (7881, ...","[4299, 7881, 62, 22866, 62, 27171, 1574, 7, 94...",140
1,"def parse_iso_8601_time_str(time_str):\n """"...","[('def', 'def', 'function_definition'), ('pars...","[(4299, 'def', 'function_definition'), (21136,...","[4299, 21136, 62, 26786, 62, 4521, 486, 62, 24...",292


# Softmax Normalization and Data Engineering

In [7]:
parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv',
 'hf_model': 'microsoft/CodeGPT-small-py-adaptedGPT2',
 'model_name': 'CodeGPT-small-py-adaptedGPT2_10000_',
 'callbacks': '../datax/callbacks-CodeGPT-small-py-adaptedGPT2_10000_',
 'wpe': 2048,
 'outputs': '../datax/outputs/out_codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv'}

In [8]:
#This code works for CodeGPT and Salesforce
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
#This code works for GPTNEO 
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [9]:
input_ids_list = tokenizer.batch_encode_plus( list(data_pd.whole_func_string.values) ) #<-- Do not return as a Tensor [cast to list]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int) for input_ids in input_ids_list.input_ids if len(input_ids) <= parameters['wpe']]

In [11]:
#Testung Sample to validate minmax functions
sample_id = 0
data_pd.iloc[sample_id].whole_func_string

'def register_context_middleware(self, *middleware):\n        """\n        :param middleware: Middleware in order of execution\n        """\n        for m in middleware:\n            if not is_generator(m):\n                raise Exception(\'Middleware {} must be a Python generator callable.\'.format(m))\n\n        self._middleware.extend(middleware)'

In [15]:
out= np.load(parameters['callbacks']+'/'+parameters['model_name']  + f'_logits_tensor[{sample_id}]_batch[{sample_id}].npy')
soft = torch.nn.Softmax( dim = 0 ) #Flattening normalization
first_token_distribution = soft( torch.from_numpy(out[0]) ) #Flattening normalization for fist token
assert round(first_token_distribution.sum().item()) == 1.0


In [10]:
tokenizer.vocab.keys()

dict_keys(['Ġveiled', 'Ġresidents', 'ĠAcid', 'iazep', 'ĠSanford', 'perhaps', 'Ġembodiments', 'Ġmaths', 'Ġasymm', '998', 'Allows', 'ĠChau', 'ĠEve', 'ĠCzech', 'high', 'Ġinvol', 'Ġaccomp', 'dec', 'Ġgoodness', 'anut', 'ĠChick', 'ĠOPEN', 'esis', 'ĠPlay', 'Ġobstacle', 'sense', 'ĠPublisher', 'ĠHick', 'ister', 'ĠPalestinians', 'Ġdistinctly', 'pload', 'ĠAS', 'Ġroutes', 'gebra', '386', '_____', '1966', 'Ġfav', 'ussion', 'haw', 'Ġdelusions', 'Dro', 'ĠMcM', 'Ġrock', 'Ġatrocities', '709', 'ĠChild', 'Ġsrc', 'Pol', 'Ġremove', 'Ġphilosopher', '×ľ', 'ĠElse', 'FFER', 'ĠDelaware', 'ĠCoff', 'Ġhomelessness', 'Relations', 'lect', 'Ġstrictly', 'Ġcommands', 'Ġfirms', 'cream', 'asionally', 'Ġclo', 'ĠLys', 'itz', 'ĠSomeone', 'ĠDVDs', 'Ġexciting', 'ista', 'Ġanybody', 'Ġenactment', 'Ġautop', 'Ġfrozen', 'Ġstripping', 've', 'hide', 'Ġsy', 'orne', 'ĠContext', 'Ġbottled', 'Ġracing', 'lishes', 'Ġbreeding', 'ĠBrazilian', 'Ġdemocracy', 'Ġsqu', 'ĠTwenty', 'crafted', 'roo', 'Madison', 'Ġhumans', 'Ġtransition', 'Ġtunnel', 

In [11]:
tokenizer

GPT2TokenizerFast(name_or_path='microsoft/CodeGPT-small-py-adaptedGPT2', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|UNKNOWN|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<pad>'})

In [12]:
tokenizer.vocab.values()

dict_values([45694, 5085, 27066, 48826, 42862, 28998, 32403, 47761, 30372, 34808, 34934, 49628, 12882, 16639, 8929, 1876, 7555, 12501, 20437, 20651, 31939, 38303, 9339, 3811, 22007, 33819, 28045, 42441, 1694, 12890, 30911, 7304, 7054, 11926, 29230, 21734, 29343, 44227, 2090, 11956, 26615, 47456, 35442, 15359, 3881, 27817, 31495, 5932, 12351, 8017, 4781, 23723, 40010, 25974, 45746, 19603, 28584, 28822, 47117, 801, 14084, 9729, 9611, 36277, 31775, 28050, 46749, 4224, 17877, 35962, 7895, 12523, 9599, 28547, 22320, 12912, 37727, 303, 24717, 827, 8553, 30532, 34123, 11717, 19724, 18954, 17036, 7996, 2809, 22381, 39160, 42407, 46845, 5384, 6801, 13275, 14738, 13375, 32756, 34018, 5807, 11291, 48143, 7607, 41264, 10766, 21897, 2524, 49651, 6425, 11331, 16256, 49235, 16181, 8970, 3738, 25336, 9774, 27545, 38452, 38586, 39300, 9131, 41482, 43996, 48948, 1019, 28292, 16675, 29623, 28831, 16205, 26998, 32571, 38902, 38329, 44134, 9305, 28064, 44810, 19377, 45960, 44325, 15922, 40868, 41167, 19199

In [13]:
tokenizer.vocab['<s>']

50258

In [14]:
tokenizer.vocab['ĠFord']

8092

In [19]:
print(tokenizer.vocab_size,out.shape[1])
assert tokenizer.vocab_size == out.shape[1]

50257 50260


AssertionError: 

In [21]:
first_token_distribution.topk(k=1, largest =True) #Returns the most probable token for that position

torch.return_types.topk(
values=tensor([0.2018]),
indices=tensor([62]))

In [23]:
def topk_tuple( logit_vocab_tensor, largest):
    "Run topk for a token"
    token_position_dict = list( tokenizer.get_vocab().keys() )
    topk = logit_vocab_tensor.topk( k=1 , largest=largest ) #TODO K number of elements can be extended
    return ( token_position_dict[topk.indices], topk.values.item() )

def min_max_logits( logit_vocab_sample_tensor ):
    "Compute min_max for a sample"
    max_cases = []
    min_cases = []
    for logit_vocab_tensor in logit_vocab_sample_tensor:
        max_cases.append( topk_tuple( logit_vocab_tensor = logit_vocab_tensor, largest = True) ) #TST Max Logit
        min_cases.append( topk_tuple( logit_vocab_tensor = logit_vocab_tensor, largest = False) ) #TST Min Logit
    return max_cases, min_cases

def actual_logit( logit_vocab_sample_tensor, sample_position, token_position_dict ):
    "Compute actual logits for a sample"
    tokenized_prompt = input_ids_list[ sample_position ]
    actual_logits_prompt = []
    for token_pos, code_token in enumerate( tokenized_prompt[1:] ): #Eliminate the first token prediction since we do not use it
        actual_logits_prompt.append(
            (   token_position_dict[int(code_token)], #retrieving the name of the token with the id
                logit_vocab_sample_tensor[token_pos][int(code_token)].item()      ) #retrieving the logit given the position in the sequence and the position in the vocab
            )
    return actual_logits_prompt

In [24]:
max_case,min_case = min_max_logits(logit_vocab_sample_tensor = [ soft( torch.from_numpy(token) ) for token in out[:4]]) #Sentence Level minmax (reduced to 3 tokens)
print(max_case)
print(min_case)

[('Ġgcc', 0.20178580284118652), ('Ġgcc', 0.5738446712493896), ('ITIES', 0.0168002936989069), ('Ġimprov', 0.42846956849098206)]
[('Ġpartitions', 4.065600395404545e-13), ('Ġlies', 5.6345922529567674e-14), ('ĠApost', 1.8752552204755364e-14), ('Ġglor', 1.3634251012402268e-15)]


In [25]:
actual_cases = actual_logit(
    logit_vocab_sample_tensor = [ soft( torch.from_numpy(token) ) for token in out] , #Out is a complete sequence
    sample_position = sample_id, #The 'sample_id' sequence from the dataset which corresponds to the out variable
    token_position_dict = list( tokenizer.get_vocab().keys()) )

In [26]:
actual_cases

[('ĠFord', 1.0120979823113885e-05),
 ('Ġgcc', 0.5738446712493896),
 ('ĠCarey', 0.0014809157000854611),
 ('Ġgcc', 0.41382652521133423),
 ('Ġsmoother', 0.0041426136158406734),
 ('Ġlegitim', 0.9843371510505676),
 ('Ġimprov', 0.7478334903717041),
 ('ĠUK', 0.21608325839042664),
 ('ĠMedium', 0.7057245373725891),
 ('dest', 0.01530979759991169),
 ('Ġsmoother', 0.3618650436401367),
 ('Ġlegitim', 0.9289371371269226),
 ('ĠGu', 0.4435915946960449),
 ('Ġpetitions', 1.3762560535646173e-11),
 ('watching', 8.918146932046511e-07),
 ('watching', 0.00047344350605271757),
 ('watching', 0.00024067176855169237),
 ('watching', 0.0012609512777999043),
 ('watching', 0.01226032804697752),
 ('watching', 0.04716210812330246),
 ('watching', 0.20471851527690887),
 ('Ġcreating', 0.00010175761417485774),
 ('Ġpetitions', 3.1926365551271374e-08),
 ('watching', 0.003879021154716611),
 ('watching', 0.00584934838116169),
 ('watching', 0.01738722249865532),
 ('watching', 0.056782759726047516),
 ('watching', 0.1209630221128

In [30]:
print(actual_cases[0])
print(actual_cases[-1])

('Ġ_', 0.001834880094975233)
('."', 0.019579626619815826)


### Sending in Batches

In [31]:
def batching_logits( size = 10000 ):
    token_position_dict = list( tokenizer.get_vocab().keys() )
    max_logit_token_prompt = []
    min_logit_token_prompt = []
    actual_logit_token_prompt = []
    soft = torch.nn.Softmax( dim = 0 )                          #Flattening normalization
    
    for file in range( size ):
        out = np.load(parameters['callbacks']+'/'+parameters['model_name']  + f'_logits_tensor[{file}]_batch[{file}].npy')
        next_tokens_distribution = [ soft( torch.from_numpy(token) ) for token in out]  #Flattening normalization
        
        max_cases,min_cases = min_max_logits(logit_vocab_sample_tensor = next_tokens_distribution)

        actual_cases = actual_logit(
            logit_vocab_sample_tensor = next_tokens_distribution,
            sample_position = file,
            token_position_dict = token_position_dict )
        
        max_logit_token_prompt.append( max_cases )
        min_logit_token_prompt.append( min_cases )
        actual_logit_token_prompt.append( actual_cases )
        
        print(file)
    return max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt

In [32]:
#max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt = batching_logits( size = 2 )
max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt = batching_logits() #<---WARNING TIME Consuming

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
dataframe_to_save = data_pd.copy()
dataframe_to_save['max_prob_case'] = max_logit_token_prompt
dataframe_to_save['min_prob_case'] = min_logit_token_prompt
dataframe_to_save['actual_prob_case'] = actual_logit_token_prompt
dataframe_to_save.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[(ers, 0.03083224408328533), ((, 0.01664721034...","[(anwhile, 1.0382577819374511e-16), (ousy, 4.6...","[(Ġ_, 0.001834880094975233), (sample, 4.047834..."
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459,"[(ers, 0.03083271160721779), (_, 0.34847137331...","[(anwhile, 1.0383171404182079e-16), (ousy, 7.3...","[(Ġcreate, 0.0009968713857233524), (_, 0.34847..."
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151,"[(ers, 0.03083224408328533), (_, 0.18502478301...","[(anwhile, 1.0382577819374511e-16), (ousy, 1.8...","[(Ġclean, 3.9070819184416905e-05), (_, 0.18502..."
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338,"[(ers, 0.030832389369606972), (_, 0.2419152259...","[(anwhile, 1.0382864354738699e-16), (ousy, 3.2...","[(Ġget, 0.002428646432235837), (_, 0.241915225..."
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185,"[(Ġde, 0.03118526190519333), (., 0.08152643591...","[(arcity, 8.036203538556996e-13), (orgetown, 4...","[(ync, 0.0002533757942728698), (Ġdef, 4.507758..."


In [34]:
parameters['outputs']

'../datax/outputs/out_codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv'

In [35]:
## Saving CheckPoint
dataframe_to_save.to_csv( parameters['outputs']  )

In [42]:
dataframe_to_save = pd.read_csv( 
                      parameters['outputs'] , 
                      index_col=0
            )

In [43]:
dataframe_to_save.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,loss
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[('ers', 0.03083224408328533), ('(', 0.0166472...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġ_', 0.001834880094975233), ('sample', 4.04...",1.089324
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459,"[('ers', 0.03083271160721779), ('_', 0.3484713...","[('anwhile', 1.0383171404182079e-16), ('ousy',...","[('Ġcreate', 0.0009968713857233524), ('_', 0.3...",1.086853
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151,"[('ers', 0.03083224408328533), ('_', 0.1850247...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġclean', 3.9070819184416905e-05), ('_', 0.1...",1.241124
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338,"[('ers', 0.030832389369606972), ('_', 0.241915...","[('anwhile', 1.0382864354738699e-16), ('ousy',...","[('Ġget', 0.002428646432235837), ('_', 0.24191...",0.793477
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185,"[('Ġde', 0.03118526190519333), ('.', 0.0815264...","[('arcity', 8.036203538556996e-13), ('orgetown...","[('ync', 0.0002533757942728698), ('Ġdef', 4.50...",1.092131


## Loss Retrieval

In [38]:
def batching_loss( size = 10000 ):
    output_loss = []
    for current_batch in range(size):
        out = np.load(parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy') 
        output_loss.append( out.item() ) #.item() for numpy library
        print(current_batch)
    return output_loss

In [39]:
output_loss = batching_loss()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
dataframe_to_save['loss'] = output_loss
dataframe_to_save.head(1)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,loss
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[('ers', 0.03083224408328533), ('(', 0.0166472...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġ_', 0.001834880094975233), ('sample', 4.04...",1.089324


In [41]:
## Saving CheckPoint 2
dataframe_to_save.to_csv( parameters['outputs']  )